In [7]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import datetime as dt
import tensorflow as tf

In [17]:
date = []
price = []
percent = []
amount = []
tprice = []
rate = []


url = 'https://finance.naver.com/sise/sise_index_day.nhn'
for i in range(650):
    param = {'code':'KOSPI', 'page':i}
    request = requests.get(url,param)

    parsing = BeautifulSoup(request.content, 'lxml')

    tag_date = parsing.select('td.date')
    tag_total = parsing.select('td.number_1')
    tag_rate = parsing.select('td.rate_down')
    tag_alt = parsing.select('img')
    
    for d in tag_date:
        d_t = d.text
        this_date=dt.datetime.strptime(d_t,"%Y.%m.%d").date()
        date.append(this_date)


    for t in range(len(tag_total)):
            if t%4==0:
                price.append(float(tag_total[t].text.replace(',', '').strip()))
    #         elif t%4==1:
    #             p = tag_total[t].text
    #             percent.append(float(re.findall("[0-9]+",p)[1].strip()))
            elif t%4==2:
                amount.append(float(tag_total[t].text.replace(',', '').strip()))
            elif t%4==3:
                tprice.append(float(tag_total[t].text.replace(',', '').strip()))
                
   

    for t in range(len(tag_rate)):
        if tag_alt[t]['alt'] == '하락':
            rate.append(-float(tag_rate[t].text.strip()))

        else:
            rate.append(float(tag_rate[t].text.strip()))

In [18]:
kospi_data = pd.DataFrame({'price':price, 'amount':amount, 'tprice':tprice, 'rate':rate}, index=date)

In [19]:
h1_node = 300
h2_node = 200
h3_node = 100
epoch = 10
a = 0.0002

In [20]:
kospi_data = kospi_data.sort_index()
kospi_data.dropna()
kospi_data = kospi_data

tf.reset_default_graph()
inputX = []
outputY = []

for i in range(len(kospi_data)):
    inputX.append(list(kospi_data.iloc[i]))
    tempY = []
    tempY.append(kospi_data.iloc[i][0])
    
    outputY.append(tempY)

x = tf.placeholder(tf.float32, shape=[None, 4], name = 'x')
y = tf.placeholder(tf.float32, shape=[None, 1], name = 'y')

testx = inputX[-1:]



W_h1 = tf.Variable(tf.truncated_normal([4, h1_node]), dtype=tf.float32, name='W_h1')
B_h1 = tf.Variable(tf.zeros([h1_node]), dtype=tf.float32, name='B_h1')

# Hidden layer의 출력값
H1 = tf.nn.relu(tf.matmul(x, W_h1) + B_h1, name='H1')
H1 = tf.nn.dropout(H1,0.7)

W_h2 = tf.Variable(tf.truncated_normal([h1_node, h2_node]), dtype=tf.float32, name='W_h2')
B_h2 = tf.Variable(tf.zeros([h2_node]), dtype=tf.float32, name='B_h2')

# Hidden layer의 출력값
H2 = tf.nn.relu(tf.matmul(H1, W_h2) + B_h2, name='H2')
H2 = tf.nn.dropout(H2,0.7)


W_h3 = tf.Variable(tf.truncated_normal([h2_node, h3_node]), dtype=tf.float32, name='W_h3')
B_h3 = tf.Variable(tf.zeros([h3_node]), dtype=tf.float32, name='B_h3')

# Hidden layer의 출력값
H3 = tf.nn.relu(tf.matmul(H2, W_h3) + B_h3, name='H3')
H3 = tf.nn.dropout(H3,0.7)


W_o = tf.Variable(tf.truncated_normal([h3_node, 1]), dtype=tf.float32, name='W_o')
B_o = tf.Variable(tf.zeros([1]), dtype=tf.float32, name='B_o')

# Output layer의 출력값
predY = tf.add(tf.matmul(H3, W_o), B_o, name='predY')

loss = tf.reduce_mean(tf.square(tf.subtract(predY, y)))
optimize = tf.train.AdamOptimizer(a).minimize(loss)



with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    

    for i in range(epoch):
        
        sess.run(optimize, feed_dict={x:inputX[:-1], y:outputY[1:]})
        cost = sess.run(loss, feed_dict={x:inputX[:-1], y:outputY[1:]})
        if i % 1000 == 0:
            print(cost)
        
        
        close = sess.run(predY, feed_dict={x:testx})
    print(close[0][0]*100)

W0705 08:57:12.431466  3432 deprecation.py:506] From <ipython-input-20-c4b3584aa6f5>:28: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


1.1971549e+19
-835464038400.0


In [ ]:
num_classes = 6
n_features = 6
hidden_size = 6
len_sequence = 6

In [ ]:
X = tf.placeholder(
    tf.float32, [None, len_sequence, n_features])  # X one-hot
Y = tf.placeholder(tf.int32, [None, len_sequence])

In [ ]:
cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)

initial_state = cell.zero_state(batch_size, tf.float32)#초기값 주는 것 
outputs, _states = tf.nn.dynamic_rnn(
    cell, X, initial_state=initial_state, dtype=tf.float32)

In [ ]:
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(
    inputs=X_for_fc, num_outputs=num_classes, activation_fn=None)

In [ ]:
learning_rate=0.01

In [ ]:
outputs = tf.reshape(outputs, 
                     [batch_size, len_sequence, num_classes])

weights = tf.ones([batch_size, len_sequence])
sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
cost = tf.reduce_mean(sequence_loss)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

prediction = tf.argmax(outputs, axis=2)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(epoches):
        l, _ = sess.run([cost, optimizer], 
                        feed_dict={X: X_one_hot, Y: y_data})
        result = sess.run(prediction, feed_dict={X: X_one_hot})
        print("{}epoch-> loss:{}".format(i, l))

        # 1 2 3 4 5 3 -> r o n m a n
        result_str = [idx2char[c] for c in np.squeeze(result)]
        